In [ ]:
import os
import pandas as pd
import numpy as np
import skimage
from skimage import transform
import nibabel
import os
from shutil import copyfile
copyfile(src, dst)

Centres=['KCL_T1','MAN_T1', 'NIJ_T1']

Demogs=os.path.join(filepath, '/mnt/storage/home/robert/Convnet_Classifier/EUAIMS_ventricle_deviations.csv')
Demographics=pd.read_excel(Demogs, index_col='subjects')

test_loc = '/mnt/storage/home/robert/Convnet_Classifier/Convnet_Classifier/data/Ventricles_Regression/'
train_loc = '/mnt/storage/home/robert/Convnet_Classifier/Convnet_Classifier/data/Ventricles_Regression/'

with open (test_loc+'/test_subs.txt', 'rb') as f:
    test_ids = f.readlines()
with open (train_loc+'/train_subs.txt', 'rb') as f:
    train_ids = f.readlines()
    
    
#Copy files to Convnet location
for centre in Centres:
    filepath=('/mnt/storage/home/robert/AIMS/FDA_Ready/V01/'+centre)
    files = os.file.listdir(filepath)
    
    c_files = list(filter(lambda k: '_Warped.nii.gz' in k, files))

    for file in c_files:
        if file[:12] in test_ids:
            copyfile(file, test_loc)
        elif file[:12] in train_ids:
            copyfile(file, train_loc)

In [ ]:
Caudate_ROIS = list(filter(lambda k: 'Caudate' in k, ROIS))

In [ ]:
'997038807877_T1_brain_MNI_Warped.nii.gz'

In [ ]:
subjects=('/mnt/storage/home/robert/3D-Convnet-for-Alzheimer-s-Detection/data/EUAIMS_MNIWARP_Order_by_ID_Subjects.txt')
with open (subjects, 'r') as f:
    subs=f.read().splitlines()
subs=[int(x) for x in subs]
Labels = Demographics[Demographics.index.isin(subs)]
cols={'t1_group'}
Labels=Labels[cols]

ssdata_dir='/mnt/storage/home/robert/AIMS/EUAIMS_MNIWARP/Harmonized_Maps'
nldata_dir='/mnt/storage/home/robert/3D-Convnet-for-Alzheimer-s-Detection/data/TD'
aldata_dir='/mnt/storage/home/robert/3D-Convnet-for-Alzheimer-s-Detection/data/ASD'

#labeling and object formation

for file in os.listdir(ssdata_dir):
    netdata=[]                                              #will be used for numpy object
    try:
        img = nibabel.load(os.path.join(ssdata_dir, file))  # loading the image
        img = img.get_fdata()                           # accessing image array
        img = skimage.transform.resize(img, (106, 106, 120))#resizing the image to dimensions(106,106,120)
        id = file.partition('.')
        label=Labels.get_value(int(id[0]), 't1_group') #getting the label
        if (label == 1 | 3):
            labelar = np.array([1, 0])
            netdata.append([img, labelar])                          #one hot encoding and saving numpy object
            np.save(os.path.join(nldata_dir, id[0] ), netdata)
        elif (label == 2 | 4):
            labelar = np.array([0, 1])
            netdata.append([img, labelar])
            np.save(os.path.join(aldata_dir, id[0] ), netdata)
    except:
        continue

#normalisation

totalnum=[]         #total number of pixels in the image
mean=[]             #mean of the pixels in the image
nummax=[]           #maximum value of pixels in the image
for file in os.listdir(aldata_dir):
    img = np.load(os.path.join(aldata_dir,file))
    mean.append(np.mean(img[0][0]))
    totalnum.append((img[0][0].shape[0]*img[0][0].shape[1]*img[0][0].shape[2]))
    nummax.append(np.max(img[0][0]))
for file in os.listdir(nldata_dir):
    img = np.load(os.path.join(nldata_dir, file))
    mean.append(np.mean(img[0][0]))
    totalnum.append((img[0][0].shape[0]*img[0][0].shape[1]*img[0][0].shape[2]))
    nummax.append(np.max(img[0][0]))
nummean=np.vdot(mean,totalnum)/np.sum(totalnum)           #mean value for the full dataset
nummax=np.max(nummax)                                     #max value for the full dataset

for file in os.listdir(aldata_dir):
    img = np.load(os.path.join(aldata_dir,file))
    img[0][0]=(img[0][0]-nummean)/nummax                 #normalisation(x-mean/max value)
    np.save(os.path.join(aldata_dir,file),img)
for file in os.listdir(nldata_dir):
    img = np.load(os.path.join(nldata_dir, file))
    img[0][0] =(img[0][0] - nummean) / nummax
    np.save(os.path.join(nldata_dir,file),img)